Here I try to train a distilbert-base-uncased bert classification model on the twittes dataset. The classification is for the 20 most common emojis in sarcastic comments.

In [ ]:
! gdown 1sxYXc7gy1ZaYW-L0geDXunEMZ3h9ixSl

Downloading...
From: https://drive.google.com/uc?id=1sxYXc7gy1ZaYW-L0geDXunEMZ3h9ixSl
To: /content/twittes.json
100% 28.7M/28.7M [00:00<00:00, 73.2MB/s]


In [ ]:
! pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=b6cb352066d59ad10ee3fbee7091e119ec10161e7d93e415bfa163f3e2f20a60
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
import json

f = open('twittes.json')
twitter_data = json.load(f)
twitter_data = twitter_data['_default']
f.close()

In [ ]:
import re

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def clean_com(com):
  com = com.lower()
  com = re.sub(r'http\S+', '', com)
  com = re.sub("@[A-Za-z0-9_]+","", com)
  com = re.sub("#[A-Za-z0-9_]+","", com)
  com = re.sub("rt :","", com)
  com = re.sub("^ +","", com)
  com = re.sub(" +$","", com)
  com = re.sub("\n","", com)
  com = re.sub(" +"," ", com)
  return remove_emoji(com)

In [ ]:
import emoji
emojis = ['😂','🤣','🙄','🤔','😉','😒','😜','😆','😁','😅','😏','👏','❤️','😎','👍','🙃','🤪','😝','😭','😳']

In [ ]:
tweets = [x['data'] for x in twitter_data.values()]
print(len(tweets))

tweets = [x for x in tweets if len(x['emojis']) == 1]
print(len(tweets))

tweets = [(x['fullText'],x['emojis'][0]) for x in tweets]
tweets = list(dict.fromkeys(tweets))
print(len(tweets))

for i in range(len(tweets)):
  tweets[i] = (clean_com(tweets[i][0]), emoji.emojize(tweets[i][1])) 

tweets = [x for x in tweets if 'sarcasm' not in x[0]]
print(len(tweets))

tweets = [x for x in tweets if 'sarcastic' not in x[0]]
print(len(tweets))

tweets = [x for x in tweets if x[1] in emojis]
print(len(tweets))


99617
71723
60393
59546
59163
36536


In [ ]:
import numpy as np

def get_one_hot_encoding(x):
  a = np.zeros(20)
  a[x] = 1
  return a

In [ ]:
X = []
Y = []
emo_dict = {}
for x,y in tweets:
    X.append(x)
    Y.append(get_one_hot_encoding(emojis.index(y)))
    if y not in emo_dict: emo_dict[y] = [x]
    else: emo_dict[y].append(x)


In [ ]:
for e in emojis:
  print(e,len(emo_dict[e]))

😂 11045
🤣 3067
🙄 7031
🤔 2412
😉 1776
😒 1481
😜 1070
😆 662
😁 901
😅 720
😏 1013
👏 504
❤️ 1174
😎 622
👍 673
🙃 856
🤪 372
😝 294
😭 420
😳 443


In [ ]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 3.9 MB/s 
     |████████████████████████████████| 6.6 MB 42.0 MB/s 
     |████████████████████████████████| 596 kB 78.8 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 68.7 MB/s 
     |████████████████████████████████| 212 kB 76.8 MB/s 
     |████████████████████████████████| 140 kB 78.0 MB/s 
     |████████████████████████████████| 127 kB 77.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 80.0 MB/s 
     |█████████████

In [ ]:
from datasets.dataset_dict import DatasetDict
from sklearn.model_selection import train_test_split
from datasets import Dataset

x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.8,test_size=0.2,random_state=42)

d = {'train':Dataset.from_dict({'label':y_train,'text':x_train}),
     'test':Dataset.from_dict({'label':y_test,'text':x_test})}

dataset = DatasetDict(d)

In [ ]:
import torch
print(torch.cuda.is_available())
torch.cuda.empty_cache()

True


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import datasets


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", problem_type="multi_class_classification")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=20, problem_type="multi_class_classification").to("cuda")
training_args = TrainingArguments(output_dir="test_trainer",evaluation_strategy="epoch",
                                  num_train_epochs=3, warmup_ratio=0.2, weight_decay=0.01,
                                  per_device_train_batch_size=32,save_steps=4000)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_class_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.20.1",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggin

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "label2id"

In [ ]:
from datasets import load_metric
import tensorflow as tf

metric = load_metric("roc_auc", multi_class='ovr')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("\nlogits:")
    print(logits)
    print("\npredictions:")
    print(predictions)
    print("\nlabels:")
    print(labels)
    print("\nresult:")
    print(metric.compute(prediction_scores=predictions, references=labels))
    return metric.compute(prediction_scores=predictions, references=labels, multi_class='ovr')

In [ ]:
acc_metric = load_metric("accuracy")

def compute_acc_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("\nlogits:")
    print(logits)
    print("\npredictions:")
    print(predictions)
    print("\nlabels:")
    print(labels)
    print("\nresult:")
    print(logits)
    print(labels)
    print(acc_metric.compute(predictions=predictions, references=labels))
    return acc_metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 29228
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2742


RuntimeError: ignored

In [ ]:
trainer.predict(small_eval_dataset).metrics